## Install system dependencies

Install Docker for [Windows](https://store.docker.com/editions/community/docker-ce-desktop-windows),
[Mac](https://store.docker.com/editions/community/docker-ce-desktop-mac) or [Linux](https://docs.docker.com/install/)

Create the travis-ci-operator local configuration directory and give permissions to travis-ci-operator user id which is used in the docker container

```
sudo mkdir -p /etc/travis-ci-operator && sudo chown 1023:1023 /etc/travis-ci-operator
```

Install and authenticate with [Travis CLI](https://github.com/travis-ci/travis.rb#installation)

## (optional) Install Conda environment to run the Jupyter notebooks

This allows to run the notebook interactively, you can skip this step and continue with `Install travis-ci-operator` section, copy-pasting the relevant code and running in a terminal

Create the Conda environment

```
conda env create -f environment.yaml
```

Activate the environment, install dependencies and start jupyter lab

```
. activate travis-ci-operator
pip install bash_kernel
python -m bash_kernel.install
jupyter lab
```

Download the [quickstart notebook](https://raw.githubusercontent.com/OriHoch/travis-ci-operator/master/QUICKSTART.ipynb) and open in Jupyter Lab

## Initialize a repo for travis-ci-operator

Verify you get a token from `travis token` command (keep it private!)

In [1]:
if travis token | python -c "import sys; print(''.join(['*' for i in range(len(sys.stdin.read()))]))"
then echo OK
else echo failed to get travis token; fi

***********************
OK


In [2]:
# change these values according to the repo you want to activate travis-ci-operator for
export GITHUB_USER="hasadna"
export GITHUB_REPO="migdar-search-ui"
export BRANCH_NAME="master"

echo This step needs to run interactively outside of the Jupyter noteboo
echo
echo run the following in a new terminal to initialize ${GITHUB_USER}/${GITHUB_REPO}:
echo
echo docker run -v /etc/travis-ci-operator:/etc/travis-ci-operator \
                -it uumpa/travis-ci-operator init \`travis token\` ${GITHUB_USER}/${GITHUB_REPO} ${BRANCH_NAME}

This step needs to run interactively outside of the Jupyter noteboo

run the following in a new terminal to initialize hasadna/migdar-search-ui:

docker run -v /etc/travis-ci-operator:/etc/travis-ci-operator -it uumpa/travis-ci-operator init `travis token` hasadna/migdar-search-ui master


The deploy keys are stored here:

In [3]:
ls -lah /etc/travis-ci-operator/${GITHUB_USER}/${GITHUB_REPO}

total 16K
drwxr-xr-x 2 1023 1023 4.0K Oct 19 14:35 .
drwxr-xr-x 4 1023 1023 4.0K Oct 19 14:35 ..
-rw------- 1 1023 1023 3.2K Oct 19 14:35 id_rsa
-rw-r--r-- 1 1023 1023  744 Oct 19 14:35 id_rsa.pub


## Initialize Docker

This allows to run docker pull / push from travis

1. Create a Docker user to be used for this purpose only: https://hub.docker.com/
2. The Docker repo should be under an organization: https://hub.docker.com/organizations/
3. Create a team for this project and add the Docker user you created to this team
4. Create a Docker repo for this project and assign this team to it with write access
5. Store the Docker user credentials for travis-ci-operator


In [ ]:
# set the credentials of the dedicated user
DOCKER_USER=""
DOCKER_PASSWORD=""

docker run -v /etc/travis-ci-operator:/etc/travis-ci-operator -it uumpa/travis-ci-operator \
    init-docker `travis token` ${GITHUB_USER}/${GITHUB_REPO} ${BRANCH_NAME} ${DOCKER_USER} ${DOCKER_PASSWORD}

## Allow to push to another repo

e.g. to update downstream

In [4]:
# set a short name for the deploy key
export DEPLOY_KEY_NAME=migdar-k8s

echo This step needs to run interactively outside of the Jupyter noteboo
echo
echo run the following in a new terminal to add deploy key ${DEPLOY_KEY_NAME}:
echo
echo docker run -v /etc/travis-ci-operator:/etc/travis-ci-operator \
                -it uumpa/travis-ci-operator add-deploy-key \`travis token\` \
                ${GITHUB_USER}/${GITHUB_REPO} ${BRANCH_NAME} ${DEPLOY_KEY_NAME}

This step needs to run interactively outside of the Jupyter noteboo

run the following in a new terminal to add deploy key migdar-k8s:

docker run -v /etc/travis-ci-operator:/etc/travis-ci-operator -it uumpa/travis-ci-operator add-deploy-key `travis token` hasadna/migdar-search-ui master migdar-k8s


## Generate .travis.yml file

Outputs an basic .travis.yml file which you can modify and extend

#### Modify travis.yml for common use-cases

Add a `.travis.sh` script to the repo:

In [5]:
DOCKER_IMAGE="uumpa/hasadna-migdar-internal-search-ui"
DEPLOY_BRANCH="master"

echo
echo Create the following files in the root of the ${GITHUB_USER}/${GITHUB_REPO} repo, ${BRANCH_NAME} branch:
echo
echo
docker run -v /etc/travis-ci-operator:/etc/travis-ci-operator -it uumpa/travis-ci-operator \
    get-travis-yml ${GITHUB_USER}/${GITHUB_REPO} ${BRANCH_NAME}

echo Create the above .travis.yml and append the following lines:
echo '
script:
- chmod +x .travis.sh && ./.travis.sh script
deploy:
  provider: script
  script: chmod +x .travis.sh && ./.travis.sh deploy
  on:
    tags: false
    condition: "$TRAVIS_BRANCH = '${DEPLOY_BRANCH}'
'

echo
echo
echo -- .travis.sh:
echo
echo '
#!/usr/bin/env bash

DOCKER_IMAGE='${DOCKER_IMAGE}'

[ -e .travis.banner ] && cat .travis.banner

if [ "${1}" == "script" ]; then
    docker build -t ${DOCKER_IMAGE}:latest -t ${DOCKER_IMAGE}:${TRAVIS_COMMIT} && exit 0
elif [ "${1}" == "deploy" ]; then
    docker push ${DOCKER_IMAGE}:latest && docker push ${DOCKER_IMAGE}:${TRAVIS_COMMIT} && exit 0
fi; exit 1
'


Create the following files in the root of the hasadna/migdar-search-ui repo, master branch:


Cloning git repo git@github.com:hasadna/migdar-search-ui.git
Cloning into '/tmp/tmp.ilKoHO'...
remote: Enumerating objects: 21, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 45 (delta 10), reused 15 (delta 4), pack-reused 24
Receiving objects: 100% (45/45), 127.16 KiB | 307.00 KiB/s, done.
Resolving deltas: 100% (14/14), done.
/tmp/tmp.ilKoHO ~
Traceback (most recent call last):
  File "/home/travis-ci-operator/read_yaml.py", line 18, in <module>
    print(json.dumps(get_from_dict(values, sys.argv[2:]), separators=(',', ':')))
  File "/home/travis-ci-operator/read_yaml.py", line 15, in get_from_dict
    return values[keys[0]]
KeyError: 'selfDeployKeyDecryptCmd'
travis.yml:
---
language: bash
sudo: required
env:
  global:
  - TRAVIS_CI_OPERATOR=1
  - secure: "Soe7CfrBMzbQIIrJi3yFPXWtMtk+gBpdrZrUsNpdNOfqzyiNX72CqNKxHk/BgFKjI3V